In [1]:
#LIBRARIES
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import math 
## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import pylab as P

In [2]:

embeddings_dict = {}
# vectors dimention
DIMENTION_DIC = 300
# 6 billions words
#with open("glove.6B."+str(DIMENTION_DIC)+"d.txt", 'r', encoding="utf-8") as f:
# 42 Billion words
with open("glove.42B."+str(DIMENTION_DIC)+"d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        print(word)
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

ney/redi-meds
stalwartuk
st-amant
stamp-size
stanchorage
s.t.a.n.d.
standord
starfox64
starle
starmind
star-san
starselite
starsports
starspow
starsshineonme
start/continue
start.html
starthtml
star-type
starvox
starwagon
stasso
state10
statec
statementlicense
state.now
state-reported
states10/10/2012
statesquick
statesthere
states.washington
state_type
stateview
stathams
station/bryan
stationthis
statkevich
statsanswersboardvideos
statti
stay-awake
staybeautiful
stboise
std100
stduluth
steamfunk
steapon
steece
steelers/ravens
steelroots
stefanibags.com
steinhour
stellenbosh
step.if
stepkother
stepo
stepsjoincontact
step-step
stepups
steri-fab
sterioids
sterlinglogin
sternn
steroided
stevn
stgc
sthd
stnto
stock11
stock.estimated
stockfinster
stockholme
stockpickers
stockpiler
stocks.com
stockwarning
stocs
stokeham
stoke-on
stolder
stommelen
stone-covered
stooop
stopgame
stop/limit
storandt
storesbusinesses
stormclan
störmer
storm-relative
storyfebruary
story?follow
storywriters
stossel

In [3]:
## ordena la spalabras dependiendo de la distancia euclidiana en el n-space
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

## method to get word/vector from dictionary
def getVectorWord(dictionary, word):
    if word in dictionary:
        return dictionary[word]
    else :
        return np.zeros(DIMENTION_DIC)

In [4]:
C1 = ["didactis", "educational assessment", "pedagogical","pre-school", "teachers", "primary", "class", "teacher training", "instructors"]

C2 = ['audio-visual techniques and media production','literature','linguistics','foreign languages','classical languages','interpretation','languages','philosophy ','ethics','history','archaeology','religion','theology','musicology','choreography','history of film and theatre','drama','music and performing arts','fine arts','audio-visual techniques','media production','fashion, interior and industrial design']

C3=['economics',
 'political sciences ',
 'civics',
 'psychology',
 'sociology ',
 'cultural studies',
 'journalism ',
 'reporting',
 'library, information ',
 'archival studies']


C4 = ['business',
 'administration',
 'finance',
 'banking',
 'insurance',
 'management',
 'administration',
 'marketing ',
 'advertising',
 'secretarial',
 'office work',
 'wholesale ',
 'retail sales',
 'work skills',
 'laws']
C5 = ['biology',
 'biochemistry',
 'environmental sciences',
 'natural environments',
 'wildlife',
 'chemistry',
 'earth sciences',
 'physics',
 'mathematics',
 'statistics']

C6 = ['information ',
 'communication technologies',
 'computer use',
 'database',
 'network design ',
 'software',
 'applications development',
 'analysis',
 'information and communication technologies ',
 'artificial intelligence']

C7 = ['nanotechnology',
 'manufacturing and processing',
 'food processing',
 'materials (glass, paper, plastic and wood)',
 'textiles (clothes, footwear and leather)',
 'architecture',
 'construction',
 'building',
 'civil engineering',
 'engineering',
 'engineering trades',
 'chemical engineering',
 'processes',
 'environmental protection technology',
 'electricity',
 'energy',
 'electronics',
 'automation',
 'mechanics',
 'metal trades',
 'motor vehicles, ships',
 'aircraf',
 'textiles (clothes, footwear and leather)']


C8 = ['agriculture',
 'crop',
 'livestock production',
 'horticulture',
 'forestry',
 'fisheries',
 'veterinary']


C9 = ['physiotherapy',
 'social work and counselling',
 'child care and youth services',
 'welfare',
 'pharmacy',
 'speech therapy',
 'nutrition/dietetics',
 'medical massage',
 'dietician programmes',
 'rehabilitation',
 'physiotherapy',
 'therapy and rehabilitation',
 'medical diagnostic and treatment technology',
 'nursing and midwifery',
 'orthodontics',
 'odontology',
 'dental assisting',
 'medical science',
 'gynaecology',
 'gerontology',
 'forensic medicine',
 'anaesthetics',
 'nursing and midwifery',
 'medicine',
 'dental studies']

C10 = ['domestic ',
 'hair',
 'beauty',
 'hotel',
 'restaurants',
 'catering',
 'sports',
 'travel, tourism',
 'leisure',
 'hygiene',
 'occupational health services',
 'occupational health',
 'safety',
 'military',
 'defence',
 'protection of persons',
 'property',
 'transport services']


In [12]:
#to vectors 
nlp = spacy.load("en_core_web_sm")

def cleaning(text):
    ##text = text.lower()
    #text = text.replace('[>/,\.!?\-!?\n\)\(\r]', ' ') 
    text = re.sub("[^A-Za-z']+", ' ', str(text))
    doc = nlp(text)
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    #txt = [token.lemma_ for token in doc if not token.is_stop]
    txt = [tok.lemma_.lower() for tok in doc
                if not tok.is_stop and not tok.is_punct and tok.is_alpha ]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    return txt

from deep_translator import GoogleTranslator
def translateString(to_translate, languageFinal):
    valid =isNaN(to_translate)
    print(valid)
    translated = GoogleTranslator(source='auto', target=languageFinal).translate("no" if valid else to_translate)
    return translated
def tranlateColum(dataFrame,columName,languageFinal):
    ## dataFrame Lengt
    total = dataFrame.shape[0]
    for i in range(0 ,total):
        to_translate = dataFrame[columName][i]
        translated = translateString(to_translate,languageFinal)
        print(i,to_translate,translated)
        dataFrame[columName][i] = None if translated == "no" else translated
def isNaN(testS):
    return not type(testS) == str and  math.isnan(testS)==True
def vectorCon(arrayStrings, label):
    totalC = []
    for word in arrayStrings:
        clean = translateString(word, "en")
        cleaned = cleaning(clean)
        nVectors = [np.zeros(DIMENTION_DIC)]
        if len(cleaned) > 0:
            nVectors  = [ getVectorWord(embeddings_dict,word) for word in cleaned]
        ##sumVector = np.add.reduce(nVectors)
        print(len(nVectors))
        totalC.append(nVectors)
    print(label,len(totalC))
    return totalC
def vectorConSingleWord(word_to:str, label:str):
    clean = translateString(word_to, "en")
    cleaned = cleaning(clean)
    nVectors = [np.zeros(DIMENTION_DIC)]
    if len(cleaned) > 0:
        nVectors  = [ getVectorWord(embeddings_dict,word) for word in cleaned]
    rta = np.add.reduce(nVectors)
    print(type(rta), label)
    return rta


In [6]:
c1 = vectorCon(C1,"C1")
c2 = vectorCon(C2,"C2")
c3 = vectorCon(C3,"C3")
c4 = vectorCon(C4,"C4")
c5 = vectorCon(C5,"C5")
c6 = vectorCon(C6,"C6")
c7 = vectorCon(C7,"C7")
c8 = vectorCon(C8,"C8")
c9 = vectorCon(C9,"C9")
c10 = vectorCon(C10,"C10")


False
1
False
2
False
1
False
2
False
1
False
1
False
1
False
2
False
1
C1 9
False
<ipython-input-5-1c1fe0307416>:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.add.reduce(totalC)
5
False
1
False
1
False
2
False
2
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
3
False
1
False
3
False
2
False
3
False
2
False
4
C2 22
False
1
False
2
False
1
False
1
False
1
False
2
False
1
False
1
False
2
False
2
C3 10
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
2
False
1
False
2
False
2
False
1
C4 15
False
1
False
1
False
2
False
2
False
1
False
1
False
2
False
1
False
1
False
1
C5 10
False
1
False
2
False
2
False
1
False
2
False
1
False
2
False
1
False
3
False
2
C6 10
False
1
False
2
False
2
F

In [14]:
allArrays = [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10]
print(vectorConSingleWord("ingeniero aeronautico", "label"))

False
<class 'numpy.ndarray'> label
[ 0.03699999 -0.097384   -0.58343     0.66715     0.23605    -0.32992
 -1.59437    -0.72085     0.438503    0.94953    -0.32368958 -0.375162
  0.11143999  0.12295599  0.23832999  0.81125     0.257314   -1.6904199
  0.233796    0.18479499  0.14330299  0.62970996  0.52517     0.358785
 -0.410726   -0.73746    -0.38225     0.381918    0.45542002 -0.18631001
 -0.56048     0.05880001 -0.67063    -0.359717    0.496128   -1.182
  0.22289997  0.5205742  -0.0393261   0.12530799 -0.420041   -0.49693
  0.29062998  0.38008898 -0.64746004 -0.32949     0.329052   -0.09372002
  0.65062     0.06797999 -0.05454    -0.04516     0.36973152  0.48366
 -0.44154802 -0.04422     0.10244099 -0.40833    -0.79849696 -0.58096004
  0.317006   -0.52776    -1.2035301   0.23839    -0.46052     0.66612405
  0.17961     1.53031    -0.55465     0.01866001  0.36324003  0.359896
 -0.68824    -0.248717    0.230387   -0.24518801  0.365215   -1.32665
 -0.566634   -0.241813   -0.17453    -1

In [19]:
def calculateAverageDistanceCategory(cateroryVectors, word_vector):
    total_distance = 0
    for vec in cateroryVectors: 
        dist = np.linalg.norm(vec-word_vector)
        total_distance += dist
    return total_distance/len(cateroryVectors)

def getCategory(education_title):
    en_edu = translateString(education_title,"en")
    vector = vectorConSingleWord(en_edu, en_edu)
    minDist = None
    index = None
    for i in range(len(allArrays)):
        print("in",i)
        categoryV = allArrays[i]
        mean_dist = calculateAverageDistanceCategory(categoryV,vector)
        print("in",mean_dist)
        if minDist == None:
            minDist = mean_dist
            index = i
        elif minDist >mean_dist :
            minDist = mean_dist
            index = i
    print(en_edu,"C"+str(index))
    return "C"+str(index)


In [25]:
getCategory("recursos humanos")

False
False
<class 'numpy.ndarray'> human Resources
in 0
in 11.428198228529764
in 1
in 11.55507723490397
in 2
in 10.840259892599923
in 3
in 11.191266695658365
in 4
in 11.050888061523438
in 5
in 10.492698052350212
in 6
in 11.74816732628401
in 7
in 11.718953490257263
in 8
in 11.73000186139887
in 9
in 11.373583475748697
human Resources C5


'C5'